
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:
import csv 
import numpy as np
# Part 1
#
# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.
#
# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.
#
# Make sure that all your data is numpy arrays and not pandas dataframes or series.
#
# Put the resulting arrays into the variables $X$ and $Y$. 
# Split the data into three parts, train/test/validation where train is 60% of the data, 
# test is 15% and validation is 25% of the data. Do not do this randomly, this is to make 
# sure that we all did the same splits (we are in this case assuming the data is IID as presented
# in the dataset). That is [train,test,validation] is the splitting layout.

'''
import chardet
with open('data/Corona_NLP_train.csv', 'rb') as rawdata:
    print(chardet.detect(rawdata.read(1000000)))
'''

with open('data/Corona_NLP_train.csv', newline='', encoding='latin-1') as csvfile:
    corona_reader = csv.reader(csvfile, delimiter=',', quotechar='"')    
    offset = 1
    X = list()
    Y = list()

    # move past the "offset", to where the data starts
    for i in range(offset):
        corona_reader.__next__()

    for row in corona_reader:
        if len(row) > 5:
            if 'Positive' in row[5]:
                    X.append(row[4])
                    Y.append(1)
            if 'Negative' in row[5]:
                    X.append(row[4])
                    Y.append(0)

X = np.array(X)
Y = np.array(Y)

len_train = np.int64(np.round(0.6*len(Y)))
len_test  = np.int64(np.floor(0.15*len(Y)))

X_train = X[0 : len_train]
Y_train = Y[0 : len_train]
X_test  = X[len_train : len_train + len_test]
Y_test  = Y[len_train : len_train + len_test]
X_valid = X[len_train + len_test :]
Y_valid = Y[len_train + len_test :]

print(type(X))
print(type(X[0]))
print(type(X[:5]))
print(X.shape, Y.shape)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape, X_valid.shape, Y_valid.shape)

<class 'numpy.ndarray'>
<class 'numpy.str_'>
<class 'numpy.ndarray'>
(33444,) (33444,)
(20066,) (20066,) (5016,) (5016,) (8362,) (8362,)


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression  
from sklearn.pipeline import Pipeline
# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

# Create a wordlist of all words in the dataset. They are made to lower case to match the 
# tokenizer of the CountVectoriser.
all_tweets = " ".join(X)
word_list = set(all_tweets.lower().split())

# Create pipeline of vectoriser and logistic regression model
pipe = Pipeline([('vectorizer', CountVectorizer()), ('log_reg', LogisticRegression())])

# Pipeline parameters, use wordlist as vocabulary for all X-data in pipeline
# set log_reg to use saga solver with max 1000 iterations.
pipe.set_params(vectorizer__vocabulary=word_list,
                log_reg__solver='saga', 
                log_reg__max_iter=1000
                )
# Train the pipeline on the training data
pipe.fit(X_train, Y_train)

# Use pipe as model
model = pipe


In [4]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.
# precision = TP / (TP+FP)
# recall    = TP / (TP+FN)

predictions = model.predict(X_test)

TP_0 = FP_0 = FN_0 = 0
TP_1 = FP_1 = FN_1 = 0

for i in range(len(predictions)):
    if predictions[i] == Y_test[i] and Y_test[i] == 0:
        TP_0 += 1
    if predictions[i] == 1 and Y_test[i] == 0:
        FN_0 += 1
        FP_1 += 1
    if predictions[i] == 0 and Y_test[i] == 1:
        FP_0 += 1
        FN_1 += 1
    if predictions[i] == Y_test[i] and Y_test[i] == 1:
        TP_1 += 1
        

precision_0 = TP_0 / (TP_0 + FP_0)
precision_1 = TP_1 / (TP_1 + FP_1)
recall_0 = TP_0 / (TP_0 + FN_0)
recall_1 = TP_1 / (TP_1 + FN_1)

print(precision_0, precision_1, recall_0, recall_1)

'''
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
print(tn, fp, fn, tp)
print((tp / (tp+fp)), (tp / (tp+fn)))
'''

0.8563945875163684 0.8741284403669725 0.8511930585683297 0.8786425673183327


'\nfrom sklearn.metrics import confusion_matrix\ntn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()\nprint(tn, fp, fn, tp)\nprint((tp / (tp+fp)), (tp / (tp+fn)))\n'

In [5]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # * A positive tweet that is classified as negative will have a cost of 1
    # * A negative tweet that is classified as positive will have a cost of 5
    # * Correct classifications cost 0

    # Note that we are interested in average cost
    
    probs = model.predict_proba(X)
    cost = 0
    
    for p, y in zip(probs[:,1], Y):
        if p > threshold and y == 0:
            cost +=1
        elif p <= threshold and y == 1:
            cost +=5
    
    return cost/len(probs)


In [6]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

from scipy.optimize import minimize_scalar

def optimize_cost(threshold, model, X, Y):
    return cost(model, threshold, X, Y)

result = minimize_scalar(optimize_cost, np.array(0.2), args=(model, X_test, Y_test), bounds=(0,1), )

optimal_threshold = result.x
cost_at_optimal_threshold = result.fun

print(optimal_threshold, cost_at_optimal_threshold)

'''
Print the cost function to spot any local minima that might mess with the optimiser
Use any visual global minima as the second argument to minimize scalar (the x0 argument)
res = np.array([cost(model, i/100, X_test, Y_test) for i in range(0,100)])
xs = np.array(range(0,100))
#print(xs)
import matplotlib.pyplot as plt
fig = plt.figure()
#print(res)
plt.plot(xs,res.reshape(-1))
plt.show()
'''

0.22813889508425328 0.27472089314194575


'\nPrint the cost function to spot any local minima that might mess with the optimiser\nUse any visual global minima as the second argument to minimize scalar (the x0 argument)\nres = np.array([cost(model, i/100, X_test, Y_test) for i in range(0,100)])\nxs = np.array(range(0,100))\n#print(xs)\nimport matplotlib.pyplot as plt\nfig = plt.figure()\n#print(res)\nplt.plot(xs,res.reshape(-1))\nplt.show()\n'

In [7]:

# Part 6
# Confidence interval
# 1-alpha = e^(-2nc^2 / (b-a)^2)
# 0.01 = e^(-2nc^2 / 5^2)
# ln(0.01) = -2nc^2 / 25
# -25ln(0.01)/2n = c^2
# c = sqrt(-25ln(0.01)/2n)
# (cost - c, cost + c)

c = np.sqrt(-25*np.log(0.01)/ (2*len(X_valid)))
print(c)

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
cost_interval_valid = (cost_at_optimal_threshold_valid - c, cost_at_optimal_threshold_valid + c)

print(cost_at_optimal_threshold_valid)
print((cost_at_optimal_threshold_valid - c, cost_at_optimal_threshold_valid + c))

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

0.08297032038145498
0.252212389380531
(np.float64(0.169242068999076), np.float64(0.33518270976198594))


In [8]:

# Part 7

costs = np.zeros(len(Y_valid))
probs = model.predict_proba(X_valid)
for i, p, y in zip(range(len(costs)), probs[:,1], Y_valid):
    ft = 1 if p > optimal_threshold else 0
    costs[i] = (1-ft)*y + 5*(1-y)*ft

c_avr = np.mean(costs)
c_var = np.mean(np.power((costs-c_avr),2))

n = len(X_valid)
b = np.max(costs) - np.min(costs)
import scipy.optimize as so
h = lambda u: (1+u)*np.log(1+u)-u
f = lambda epsilon: np.exp(-n*c_var/b**2*h(epsilon/c_var))-0.01
ans = so.fsolve(f,0.00001)

variance_of_C = c_var
interval_of_C = (c_var-ans[0], c_var+ans[0])

'''
import matplotlib.pyplot as plt
fig = plt.figure()
xs = np.array(range(-int(c_var*20),int(c_var*20)))
fs = f(xs/100)
plt.plot(xs/100, fs)
plt.show()
'''
print(c_var)
print(ans)
print((c_var-ans[0], c_var+ans[0]))

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

3.3810902311403948
[-0.3005026]
(np.float64(3.681592829854102), np.float64(3.0805876324266874))
